In [1]:
import pandas as pd
import numpy as np
from screed import ScreedDB

In [ ]:
metadata = pd.read_csv("/scratch/jho5ze/bionets/covid/variant_data/metadata.tsv", sep="\t", parse_dates = ["Collection date", "Submission date"])
# print(metadata.columns)
# metadata = metadata[["Accession ID", \
#                    "Collection date", \
#                    "Submission date", \
#                    "Location", \
#                    "Additional location information", \
#                    "Sequence length", \
#                    "Host", \
#                    "AA Substitutions", \
#                    "Is reference?", \
#                    "Pango lineage"]]
def get_nth_slash(row, n):
    try:
        return row.split("/")[n].strip()
    except:
        return np.nan
    
for i in range(4):
    metadata[f"Location_{i}"] = metadata["Location"].apply(lambda row: get_nth_slash(row, i))
metadata = metadata.rename(columns={"Location_1":"country", "Location_2":"state"})
metadata = metadata[metadata["country"] == "USA"]
metadata["alias"] = metadata["Virus name"].apply(lambda row: "/".join(row.split("/")[2:]))
# metdata = metadata.rename(columns={"Accession ID":"EPI"})